In [1]:
# Benders decomposition for Fixed Charge Transportation Problem

import gurobipy as gp
from gurobipy import *
import numpy as np

In [2]:
capacity = [5,8,7]
demand = [3,6,6,5]

locations = len(capacity)
customers = len(demand)

varcost = [[4,5,2,7],[5,8,6,2],[8,9,4,3]]
fixcost = [[2,10,1,5],[7,2,7,8],[4,0,3,9]]

In [3]:
bigM = [[min(capacity[i], demand[j]) for j in range(customers)] for i in range(locations)]
bigM

[[3, 5, 5, 5], [3, 6, 6, 5], [3, 6, 6, 5]]

In [50]:
# Define Benders subproblem
        
def subproblem(model, where):
    if where == GRB.Callback.MIPSOL:
        
        v_y = model.cbGetSolution(model._y)
        print('\nCurrent y: ', v_y)
        
        LB = model.cbGetSolution(model._z)
        print('Current LB: ', LB, '\n')
        
        
        bsp = gp.Model("Subproblem")
        v = {}
        for j in range(customers): v[j] = bsp.addVar(obj = demand[j])
        w = {}
        for i in range(locations):
            for j in range(customers):
                w[i,j] = bsp.addVar(obj = - v_y[i,j] * bigM[i][j])
        u = {}
        for i in range(locations): u[i] = bsp.addVar(obj = - capacity[i])
        
        for i in range(locations):
            for j in range(customers):
                bsp.addConstr( v[j] - u[i] - w[i,j] <= varcost[i][j]);
        
        bsp.modelSense = -1
        bsp.optimize()
        
        print('\n\n',bsp.Status,'\n\n')
        
        if bsp.Status not in [3,4,5]: #infeasible or unbounded subproblem
            if bsp.objVal > LB:
                cm = np.zeros((locations,customers)) #coefficient of y in relaxed master problem
                for i in range(locations):
                    for j in range(customers):
                        cm[i,j] = fixcost[i][j] - bigM[i][j] * w[i,j].x

                model.addConstr(model._z <= min(model._z.ub, 
                                  sum(fixcost[i][j]*v_y[i,j] for i in range(locations) for j in range(customers)) + bsp.objVal))
                model.cbLazy(model._z >= sum(demand[j] * v[j].x for j in range(customers)) - 
                             sum(capacity[i] * u[i].x for i in range(locations)) + 
                             sum(cm[i,j] * model._y[i,j] for i in range(locations) for j in range(customers)))
                
                
                print('\n\n')
                print(sum(fixcost[i][j]*v_y[i,j] for i in range(locations) for j in range(customers)) + bsp.objVal)
                print(model._z >= sum(demand[j] * v[j].x for j in range(customers)) -  
                      sum(capacity[i] * u[i].x for i in range(locations)) +    
                      sum(cm[i,j] * model._y[i,j] for i in range(locations) for j in range(customers)))
        else:          
            bsp.dispose()
            
            infbsp = gp.Model('Infeasible subproblem')
            dummy = infbsp.addVar(obj = 1)
            
            v = {}
            for j in range(customers): v[j] = infbsp.addVar()
            w = {}
            for i in range(locations):
                for j in range(customers):
                    w[i,j] = infbsp.addVar()
            u = {}
            for i in range(locations): u[i] = infbsp.addVar()
            
            infbsp.addConstr( sum(demand[j]*v[j] for j in range(customers)) +  
                              sum(- capacity[i]*u[i] for i in range(locations)) +  
                              sum(- bigM[i][j]*v_y[i,j]*w[i,j] for i in range(locations) for j in range(customers)) == 1)                        
            for i in range(locations):
                for j in range(customers):
                    infbsp.addConstr( v[j] - u[i] - w[i,j] <= 0);
            infbsp.addConstr(dummy == 0)
            
            print(infbsp.Status)
            
            infbsp.modelSense = -1 #set to maximization
            infbsp.optimize()        
            
            #positive dual variables => use - instead of +
            model.cbLazy( sum(demand[j] * v[j].x for j in range(customers)) - 
                sum(capacity[i] * u[i].x for i in range(locations)) - 
                sum(bigM[i][j]*w[i,j].x*model._y[i,j] for i in range(locations) for j in range(customers)) <= 0)
            print(sum(demand[j] * v[j].x for j in range(customers)) - 
                sum(capacity[i] * u[i].x for i in range(locations)) - 
                sum(bigM[i][j]*w[i,j].x*model._y[i,j] for i in range(locations) for j in range(customers)) <= 0)
            
            
                       

In [51]:

# Masterproblem
                     
m = gp.Model("Benders Fixed Charge Transportation Masterproblem")

y = {}
z = m.addVar(obj=1)                                                                           
for i in range(locations):
    for j in range(customers):
        y[i,j] = m.addVar(vtype=GRB.BINARY)

for j in range(customers):
    m.addConstr(sum(y[i,j] for i in range(locations)) >= 1)
        
m.Params.lazyConstraints = 1
m._y = y
m._z = z

transport = np.zeros((locations, customers)) # to save result of x from subproblem
m.optimize(subproblem)

print("Objective: " + str(m.objVal))
print([y[i,j].x for i in range(locations) for j in range(customers)])

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 4 rows, 13 columns and 12 nonzeros
Model fingerprint: 0x118486d8
Variable types: 1 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Current y:  {(0, 0): 1.0, (0, 1): 1.0, (0, 2): 1.0, (0, 3): 1.0, (1, 0): 1.0, (1, 1): 1.0, (1, 2): 1.0, (1, 3): 1.0, (2, 0): 1.0, (2, 1): 1.0, (2, 2): 1.0, (2, 3): 1.0}
Current LB:  0.0 

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 12 rows, 19 columns and 36 nonzeros
Model fingerprint: 0x0dc12283
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 9e+00]
Presolve time: 0.00s
Presolved: 12 rows, 19 columns, 36 nonzeros



Optimize a model with 12 rows, 19 columns and 36 nonzeros
Model fingerprint: 0x79cec41e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 9e+00]
Presolve removed 8 rows and 9 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.01 seconds
Infeasible or unbounded model


 4 


1
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 14 rows, 20 columns and 48 nonzeros
Model fingerprint: 0x79c06e01
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 14 rows and 20 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
<guro

Optimize a model with 12 rows, 19 columns and 36 nonzeros
Model fingerprint: 0x725baef6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 9e+00]
Presolve removed 12 rows and 19 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.3000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  9.300000000e+01


 2 





114.0
<gurobi.TempConstr: <gurobi.LinExpr: C0> >= <gurobi.LinExpr: 93.0 + 2.0 C1 + 0.0 C2 + C3 + 5.0 C4 + 7.0 C5 + -4.0 C6 + 7.0 C7 + 8.0 C8 + 4.0 C9 + 0.0 C10 + 3.0 C11 + 9.0 C12>>

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  101.34783    0    4          -  101.34783      -     -    0s

Current y:  {(0, 0): 1.0, (

In [22]:
k = 1
mvars = m.getVars()
y0 = np.zeros((locations, customers))
for i in range(locations):
    for j in range(customers):
        y0[i,j] = mvars[k].x
        k+=1

In [23]:
model0 = gp.Model()
x0 = {}
for i in range(locations):
    for j in range(customers):
        x0[i,j] = model0.addVar(obj=varcost[i][j])

#demand constraint
demand_constr = {} 
for j in range(customers):
    demand_constr[j] = model0.addConstr(sum(x0[i,j] for i in range(locations)) >= demand[j])
        
#capacity constraint
cap_constr = {}
for i in range(locations):
    cap_constr[i] = model0.addConstr(sum(x0[i,j] for j in range(customers)) <= capacity[i])

#logical constraint
log_constr = {}
for i in range(locations):
    for j in range(customers):
        log_constr[i,j] = model0.addConstr(x0[i,j] <= y0[i][j] * min(capacity[i], demand[j]))

model0.optimize()

print("Objective: " + str(model0.objVal))

print("Solution:")
import pandas as pd
index = ['Source ' + str(x) for x in range(locations)]
columns = ['Customer ' + str(x) for x in range(customers)]

solution = pd.DataFrame(index=index, columns=columns)

for i in range(locations):
    for j in range(customers):
        solution.iloc[i,j] = x0[i,j].x

print(solution)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 19 rows, 12 columns and 36 nonzeros
Model fingerprint: 0xb4ea7b23
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 8e+00]
Presolve removed 19 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.9000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  8.900000000e+01
Objective: 89.0
Solution:
          Customer 0  Customer 1  Customer 2  Customer 3
Source 0           3           2           0           0
Source 1           0           3           0           5
Source 2           0           1           6           0


In [24]:
89 + sum(y0[i][j]*fixcost[i][j] for i in range(locations) for j in range(customers))

114.0